In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np 
import joblib,glob,os,cv2
from scipy.ndimage import filters
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import matplotlib.pyplot as plt

In [3]:
def get_gradient(im_dx, im_dy): 
    assert im_dx.shape == im_dy.shape
    m, n = im_dx.shape
    grad_mag = np.zeros((m, n))
    grad_angle = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            grad_mag[i][j] = np.sqrt(im_dx[i][j] ** 2 + im_dy[i][j] ** 2)
            grad_angle[i][j] = np.arctan2(im_dy[i][j], im_dx[i][j])
    return grad_mag, grad_angle

In [4]:
# check angle in the bin array:
def checkAngle(bins, angle):
  if angle not in bins: return -1
  if angle in bins: return bins.index(angle)

In [5]:
def get_angle_proportion(bins,direc,mag,bs): # bs is bin_size, mag is magnitude, direc is direction

    prop1=abs((bins[0]-direc))
    prop2=abs((bins[1]-direc))

    if prop1<prop2:
      prop1=(prop1/bs)*mag
      prop2=(prop2/bs)*mag
      keys=[bins[0],prop1,bins[1],prop2] # [bins[0]:prop1, bins[1]:prop2] 
    else:
      prop1=(prop1/bs)*mag
      prop2=(prop2/bs)*mag
      keys=[bins[0],prop2,bins[1],prop1]
    return keys
    

In [6]:
# get bins index for the direction:
def get_angle_bin(bins, angle):
  if angle<0:
    #print("before",angle)
    angle= angle*-1
    #print("after",angle)

  if 0 < angle < 20: 
    bins=[0,20]
    return bins
  elif 20 <= angle < 40:
    bins=[20,40]
    return bins 
  elif 40 <= angle < 60:
    bins=[40,60]
    return bins 
  elif 60 <= angle < 80:
    bins=[60,80]
    return bins 
  elif 80 <= angle < 100:
    bins=[80,100]
    return bins 
  elif 100 <= angle < 120:
    bins=[100,120]
    return bins 
  elif 120 <= angle < 140:
    bins=[120,140]
    return bins 
  elif 140 <= angle < 160:
    bins=[140,160]
    return bins 
  elif 160 <= angle <= 180:
    bins=[160,180]
    return bins 
  print("nooo",angle)

In [7]:
# calculating the histogram of Gradients in 8×8 cells

# histogram of 9 bins with 20 degrees difference: 0,20, 40, 60, 80, 100, 120, 140, 160
def build_histogram(magnitude, direction, cell_size):
  bin_size = 9
  cell_size=cell_size
  bins=[0,20, 40, 60, 80, 100, 120, 140, 160]
  
  row,col=magnitude.shape
  row = int(row / cell_size)
  col = int(col / cell_size)
  histogram=np.zeros((row,col, bin_size))
  for i in range(row):
    for j in range(col):
      for ii in range(cell_size):
        for jj in range(cell_size):
          index=checkAngle(bins,direction[i * cell_size + ii][j * cell_size + jj])
          if index!=-1:
            histogram[i][j][index]= histogram[i][j][index] + magnitude[i * cell_size + ii][j * cell_size + jj]
          else:
            angle_bins=get_angle_bin(bins,direction[i * cell_size + ii][j * cell_size + jj])
            proportion=get_angle_proportion(angle_bins,direction[i * cell_size + ii][j * cell_size + jj],magnitude[i * cell_size + ii][j * cell_size + jj],bin_size)
            index1=checkAngle(bins,proportion[0])
            index2=checkAngle(bins,proportion[2])
            if index1 == 180:
              index1= 0
            if index2 == 180:
              index2 =0
            histogram[i][j][index1]= histogram[i][j][index1] + proportion[1]
            histogram[i][j][index2]= histogram[i][j][index2] + proportion[3]
  return histogram  

In [8]:
def get_block_descriptor(ori_histo, block_size):
    M, N, bins = ori_histo.shape
    e = 0.001
    ori_histo_normalized = np.zeros((M - block_size + 1, N - block_size + 1, bins * block_size * block_size))
    for i in range(M - block_size + 1):
        for j in range(N - block_size + 1):
            unnormalized = []
            for x in range(block_size):
                for y in range(block_size):
                    for z in range(bins):
                        unnormalized.append(ori_histo[i + x][j + y][z])
            unnormalized = np.asarray(unnormalized)
            den = np.sqrt(np.sum(unnormalized ** 2) + e ** 2)
            normalized = unnormalized / den
            for p in range(bins * block_size * block_size):
                ori_histo_normalized[i][j][p] = normalized[p]
            return ori_histo_normalized

In [9]:
data = []
labels = []
human_im_path = '/content/drive/My Drive/Dataset/DATAIMAGE/human/'
nonhuman_im_path = '/content/drive/My Drive/Dataset/DATAIMAGE/nonhuman/'


In [10]:
# Load the positive features
for filename in glob.glob(os.path.join(human_im_path,"*.png")):
    im = cv2.imread(filename,0)
    im = cv2.resize(im,(64,128))
    im_dx = filters.gaussian_filter(im,(3,3),(1,0))
    im_dy = filters.gaussian_filter(im,(3,3),(0,1))
    grad_mag, grad_angle = get_gradient(im_dx, im_dy)
    ori_histo = build_histogram(grad_mag, grad_angle, 8)
    ori_histo_normalized = get_block_descriptor(ori_histo, 2)
    hog = ori_histo_normalized.reshape((-1))
    data.append(hog)
    labels.append(1)


In [11]:
im_dx.shape

(128, 64)

In [12]:
im_dy.shape

(128, 64)

In [13]:
grad_mag.shape

(128, 64)

In [14]:
grad_angle.shape

(128, 64)

In [15]:
ori_histo.shape

(16, 8, 9)

In [16]:
ori_histo_normalized.shape

(15, 7, 36)

In [17]:
len(data)

2426

In [18]:
# Load the positive features
for filename in glob.glob(os.path.join(nonhuman_im_path,"*.jpg")):
    im = cv2.imread(filename,0)
    im = cv2.resize(im,(64,128))
    im_dx = filters.gaussian_filter(im,(3,3),(1,0))
    im_dy = filters.gaussian_filter(im,(3,3),(0,1))
    grad_mag, grad_angle = get_gradient(im_dx, im_dy)
    ori_histo = build_histogram(grad_mag, grad_angle, 8)
    ori_histo_normalized = get_block_descriptor(ori_histo, 2)
    hog = ori_histo_normalized.reshape((-1))
    data.append(hog)
    labels.append(0)

In [19]:
len(data)

5660

In [20]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, 
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=0)
print("Labels for training and testing data")
print(len(train_data))
print(len(test_data))

Labels for training and testing data
3961
1698


In [21]:
print(len(train_labels))
print(len(test_labels))

3961
1698


In [22]:
hog.shape

(3780,)

In [23]:
labels = np.unique(train_labels)
print(labels) 

[0 1]


In [24]:
model = SVC(C = 1, kernel='linear')

In [25]:
model.fit(train_data, train_labels)

SVC(C=1, kernel='linear')

In [26]:
predictions = model.predict(test_data)
print(predictions)

[1 0 1 ... 1 1 0]


In [27]:
percentage = model.score(test_data, test_labels)

In [30]:
from sklearn.metrics import confusion_matrix
res = confusion_matrix(test_labels, predictions)
print("Confusion Matrix")
print(res)
print(f"Test Set: {len(test_data)}")
print(f"Accuracy = {percentage*100} %")

Confusion Matrix
[[632 340]
 [367 359]]
Test Set: 1698
Accuracy = 58.362779740871616 %
